In [3]:
import pandas as pd

df_ = pd.read_csv("data/mimic-iv-cohort.csv")
df_.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,admittime,dischtime,...,creatinine,lactate,lactate_cat,bicarbonate,hemoglobin,sodium,potassium,glucose,platelet,log_treatment
0,17195991,23542772,38319097,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2110-01-12T00:54:00Z,2110-01-14T22:07:39Z,2.884479,2110-01-11T22:47:00Z,2110-01-18T10:25:00Z,...,0.7,NaN,missing,23.0,11.6,145.0,2.8,116.0,247.0,3.108574
1,12770182,20446666,34901199,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2110-01-18T17:47:47Z,2110-01-20T22:25:09Z,2.192616,2110-01-18T17:46:00Z,2110-01-21T16:30:00Z,...,0.8,NaN,missing,28.0,14.8,142.0,4.2,114.0,300.0,2.757537
2,15173979,28605254,33198006,Trauma SICU (TSICU),Trauma SICU (TSICU),2110-02-02T20:55:57Z,2110-02-04T16:54:51Z,1.832569,2110-02-02T18:56:00Z,2110-02-08T18:30:00Z,...,1.8,1.0,<2,23.0,15.0,137.0,4.9,115.0,171.0,3.263041
3,18425495,25117070,35090086,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2110-02-03T18:04:00Z,2110-02-07T00:30:01Z,3.268067,2110-02-03T16:46:00Z,2110-02-09T14:08:00Z,...,1.2,NaN,missing,17.0,10.0,143.0,3.4,103.0,116.0,2.607134
4,10582595,20690213,33235984,Surgical Intensive Care Unit (SICU),Trauma SICU (TSICU),2110-02-09T22:11:00Z,2110-02-13T17:33:51Z,3.807535,2110-02-09T20:41:00Z,2110-02-17T15:40:00Z,...,0.7,NaN,missing,28.0,NaN,134.0,4.1,135.0,NaN,2.919146


In [11]:
df_.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los', 'admittime', 'dischtime', 'deathtime',
       'admission_type', 'admission_location', 'discharge_location',
       'insurance', 'language', 'marital_status', 'ethnicity', 'edregtime',
       'edouttime', 'hospital_expire_flag', 'total.stays', 'age', 'gender',
       'ed_icu.time.diff', 'score', 'chf', 'esrd', 'chf.esrd', 'sepsis.only',
       'subgroup', 'day1', 'patientweight', 'treatment', 'vaso', 'norep',
       'pressor', 'ventilation_status', 'sofa_start', 'sofa_end', 'sofa_delta',
       'sofa_24hours', 'DNI', 'temperature', 'heart_rate', 'wbc', 'creatinine',
       'lactate', 'lactate_cat', 'bicarbonate', 'hemoglobin', 'sodium',
       'potassium', 'glucose', 'platelet', 'log_treatment'],
      dtype='object')

In [14]:
outcomes = ["hospital_expire_flag"] #does he die or not
cohort = ["chf", "esrd"] #dummies variable
covariates = [
    "age",
    "gender",
    "temperature",
    "heart_rate",
    "wbc",
    "creatinine",
    "lactate_cat",
    "bicarbonate",
    "hemoglobin",
    "sodium",
    "potassium",
    "glucose",
    "platelet",
]
df_[outcomes]

,hospital_expire_flag
0,0
1,0
2,0
3,0
4,0
...,...
14793,0
14794,1
14795,1
14796,0
